# Data Science nel Cloud: Il metodo "Azure ML SDK"

## Introduzione

In questo notebook, impareremo a utilizzare l'Azure ML SDK per addestrare, distribuire e consumare un modello tramite Azure ML.

Prerequisiti:
1. Hai creato un workspace di Azure ML.
2. Hai caricato il [dataset Heart Failure](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) in Azure ML.
3. Hai caricato questo notebook in Azure ML Studio.

I prossimi passi sono:

1. Creare un Esperimento in un Workspace esistente.
2. Creare un cluster di Calcolo.
3. Caricare il dataset.
4. Configurare AutoML utilizzando AutoMLConfig.
5. Eseguire l'esperimento AutoML.
6. Esplorare i risultati e ottenere il miglior modello.
7. Registrare il miglior modello.
8. Distribuire il miglior modello.
9. Consumare l'endpoint.

## Importazioni specifiche per Azure Machine Learning SDK


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## Inizializzare l'area di lavoro
Inizializza un oggetto area di lavoro dalla configurazione salvata. Assicurati che il file di configurazione sia presente in .\config.json


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Creare un esperimento Azure ML

Creiamo un esperimento chiamato 'aml-experiment' nello spazio di lavoro che abbiamo appena inizializzato.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## Creare un Cluster di Calcolo  
Dovrai creare un [compute target](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) per la tua esecuzione AutoML.


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## Dati
Assicurati di aver caricato il dataset su Azure ML e che la chiave abbia lo stesso nome del dataset.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## Configurazione AutoML


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## Esecuzione AutoML


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## Distribuire il Miglior Modello

Esegui il seguente codice per distribuire il miglior modello. Puoi vedere lo stato della distribuzione nel portale di Azure ML. Questo passaggio può richiedere alcuni minuti.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## Utilizzare l'Endpoint
Puoi aggiungere input al seguente esempio di input.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali fraintendimenti o interpretazioni errate derivanti dall'uso di questa traduzione.
